# Building A Chatbot

In this notebook, we'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

> **Note:** This chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- **Conversational RAG**: Enable a chatbot experience over an external source of data  
- **Agents**: Build a chatbot that can take actions


In [1]:
## OpenAI API and Groq Key and Opensource models
import os
from dotenv import load_dotenv
import openai

load_dotenv()

openai.api_key=os.getenv("OPENAI_API_KEY")

groq_api_key=os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq
model=ChatGroq(model="gemma2-9b-it",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001BBCC300970>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001BBCC301DE0>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hello,My name is Faraz,and I am an AI Full Stack Developer.")])

AIMessage(content="Hello Faraz, it's nice to meet you! That's a very impressive title. \n\nAs an AI, I'm always interested in learning more about how people are using technology, especially in fields like full stack development. \n\nWhat kind of projects are you working on these days?  Do you have a particular area of full stack development that you specialize in?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 26, 'total_tokens': 107, 'completion_time': 0.147272727, 'prompt_time': 0.002132166, 'queue_time': 0.235544774, 'total_time': 0.149404893}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-4e4b8042-0deb-4114-a3a7-5e61c0ed9766-0', usage_metadata={'input_tokens': 26, 'output_tokens': 81, 'total_tokens': 107})

In [ ]:
## Example of Messages to LLM and whether it remember them-Yes
from langchain_core.messages import HumanMessage, AIMessage

messages = [
    HumanMessage(content="Hello,My name is Faraz,and I am an AI Full Stack Developer."),
    AIMessage(content="Hello Faraz, it's nice to meet you! That's a very impressive title. \n\nAs an AI, I'm always interested in learning more about how people are using technology, especially in fields like full stack development. \n\nWhat kind of projects are you working on these days?  Do you have a particular area of full stack development that you specialize in?"),
    HumanMessage(content="Hey what's my name and what do I do?")
]

response=model.invoke(messages)

In [5]:
response

AIMessage(content="You said your name is Faraz, and that you are an AI Full Stack Developer.  \n\nIs there anything else you'd like to tell me about yourself? 😊  I'm eager to learn more!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 126, 'total_tokens': 173, 'completion_time': 0.085454545, 'prompt_time': 0.006744446, 'queue_time': 0.235993742, 'total_time': 0.092198991}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-abb23ab7-e967-4a02-8b1b-60edb0528ef8-0', usage_metadata={'input_tokens': 126, 'output_tokens': 47, 'total_tokens': 173})

# Message History

We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let’s see how to use this!

In [11]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)


In [14]:
config = {
    "configurable": {
        "session_id": "user_faraz_001"
    }
}


In [16]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi, I am Faraz, an AI Software Engineer.")],
    config=config
)

In [17]:
response.content

"Hi Faraz, it's great to meet you!  \n\nBeing an AI Software Engineer is a fascinating field. What kind of projects are you working on these days?  \n\nDo you have any specific questions about AI or anything you'd like to discuss? I'm here to help in any way I can. \n\n"

In [19]:
## Checking if AI remembers my name
with_message_history.invoke(
    [HumanMessage(content="What is my Name?")],
    config=config
)

AIMessage(content='You said your name is Faraz.  \n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 248, 'total_tokens': 271, 'completion_time': 0.041818182, 'prompt_time': 0.009359292, 'queue_time': 0.019291805000000002, 'total_time': 0.051177474}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-0063e7bb-aea6-47f4-ad5a-74ce66c20db8-0', usage_metadata={'input_tokens': 248, 'output_tokens': 23, 'total_tokens': 271})

In [20]:
## Changing config-->session id--> To check if it still remembers
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content


"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to learn it! 😊\n"

In [ ]:
## Re-tell him you name---> now it remembers
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John! It's nice to meet you.  \n\nWhat can I do for you today?\n"